In [1]:
!pip install wandb
!pip install fairlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=ceef0ca9954fe578ec4c7e580d3350c35c9c4eca454199517c34e4118067bcf8
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 KB 4.5 MB/s eta 0:00:00


In [113]:
import pandas as pd
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model, metrics, preprocessing
from datetime import datetime, timedelta

import wandb
import random

from fairlearn.postprocessing import ThresholdOptimizer, plot_threshold_optimizer
from fairlearn.metrics import demographic_parity_ratio, equalized_odds_ratio, demographic_parity_difference, equalized_odds_difference
from fairlearn.reductions import DemographicParity


from google.colab import drive

from sklearn.model_selection import KFold, cross_val_score



drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# New Section

In [32]:
import torch
from torch import nn
from torch.nn import functional as F


class ConstraintLoss(nn.Module):
    def __init__(self, n_class=2, alpha=1, p_norm=2):
        super(ConstraintLoss, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.alpha = alpha
        self.p_norm = p_norm
        self.n_class = n_class
        self.n_constraints = 2
        self.dim_condition = self.n_class + 1
        self.M = torch.zeros((self.n_constraints, self.dim_condition))
        self.c = torch.zeros(self.n_constraints)

    def mu_f(self, X=None, y=None, sensitive=None):
        return torch.zeros(self.n_constraints)

    def forward(self, X, out, sensitive, y=None):
        # Reshapes sensitive attribute tensor to the same shape as the output 
        sensitive = sensitive.view(out.shape)
        # Reshapes y (label) tensor to the same shape as the output 
        if isinstance(y, torch.Tensor):
            y = y.view(out.shape)
        # Get probability output by applying sigmoid (like logistic regression?)
        out = torch.sigmoid(out)
        # Get the mu_f value given these tensors 
        mu = self.mu_f(X=X, out=out, sensitive=sensitive, y=y)
        # Gap constraint???
        # Apply relu to matrix vector product of M compared to mu-c 
        # Using cuda if applicable (self.device references)
        gap_constraint = F.relu(
            torch.mv(self.M.to(self.device), mu.to(self.device)) - self.c.to(self.device)
        )
        # Using the L2 Norm of the gap constraint as regularizer with alpha parameter. 
        if self.p_norm == 2:
            cons = self.alpha * torch.dot(gap_constraint, gap_constraint)
        else:
            cons = self.alpha * torch.dot(gap_constraint.detach(), gap_constraint)
        return cons


class DemographicParityLoss(ConstraintLoss):
    def __init__(self, sensitive_classes=[0, 1], alpha=1, p_norm=2):
        """loss of demograpfhic parity

        Args:
            sensitive_classes (list, optional): list of unique values of sensitive attribute. Defaults to [0, 1].
            alpha (int, optional): [description]. Defaults to 1.
            p_norm (int, optional): [description]. Defaults to 2.
        """
        self.sensitive_classes = sensitive_classes
        self.n_class = len(sensitive_classes)
        super(DemographicParityLoss, self).__init__(
            n_class=self.n_class, alpha=alpha, p_norm=p_norm
        )
        self.n_constraints = self.n_class
        self.dim_condition = self.n_class
        self.M = torch.zeros((self.n_constraints, self.dim_condition))
        for i in range(self.n_constraints):
            j = i % 2
            if j == 0:
                self.M[i, j] = 1.0
                self.M[i, 1] = -1.0
            else:
                self.M[i, j - 1] = -1.0
                self.M[i, 1] = 1.0
        # print("DP_init -> M defined")

    def mu_f(self, X, out, sensitive, y=None):
        # print("sub-class mu_f -> return exp vals")
        expected_values_list = []
        for v in self.sensitive_classes:
            # Get the index for each of the senstive classes 
            idx_true = sensitive == v  # torch.bool
            # Get the average prediction for that sensitive class
            expected_values_list.append(out[idx_true].mean())

        #expected_values_list will have two values, one for each sensitive class representing it's average prediction
        #if you look at forward you'll see we take the dot product of these two values with M
        return torch.stack(expected_values_list)

    def forward(self, X, out, sensitive, y=None):
        #Calls parent class forward function
        # print("sub-class forward -> call super forward")
        return super(DemographicParityLoss, self).forward(X, out, sensitive)


# Convex 
class EqualizedOddsLoss(ConstraintLoss):
    def __init__(self, sensitive_classes=[0, 1], alpha=1, p_norm=2):
        """loss of demograpfhic parity
        Args:
            sensitive_classes (list, optional): list of unique values of sensitive attribute. Defaults to [0, 1].
            alpha (int, optional): [description]. Defaults to 1.
            p_norm (int, optional): [description]. Defaults to 2.

            equalized odds: pr(y_hat | A=a, Y=y) = pr(y_hat | Y=y) for all a in A, y in Y.
            pr(y_hat =1 | A=a1, Y=y) = pr(y_hat =1 | A=a2, Y=y) for all y
        """
        self.sensitive_classes = sensitive_classes
        self.y_classes = [0, 1] # binary classification
        self.n_class = len(sensitive_classes)
        self.n_y_class = len(self.y_classes)
        super(EqualizedOddsLoss, self).__init__(n_class=self.n_class, alpha=alpha, p_norm=p_norm)
        # K:  number of constraint : (|A| x |Y| x {+, -})
        self.n_constraints = self.n_class * self.n_y_class * 2 
        # J : dim of conditions  : ((|A|+1) x |Y|)
        self.dim_condition = self.n_y_class * (self.n_class + 1)
        self.M = torch.zeros((self.n_constraints, self.dim_condition))
        # make M (K * J): (|A| x |Y| x {+, -})  *   (|A|+1) x |Y|) )
        self.c = torch.zeros(self.n_constraints)
        element_K_A = self.sensitive_classes + [None]
        for i_a, a_0 in enumerate(self.sensitive_classes):
            for i_y, y_0 in enumerate(self.y_classes):
                for i_s, s in enumerate([-1, 1]):
                    for j_y, y_1 in enumerate(self.y_classes):
                        for j_a, a_1 in enumerate(element_K_A):
                            i = i_a * (2 * self.n_y_class) + i_y * 2 + i_s
                            j = j_y + self.n_y_class * j_a
                            self.M[i, j] = self.__element_M(a_0, a_1, y_1, y_1, s)
    def __element_M(self, a0, a1, y0, y1, s):
        if a0 is None or a1 is None:
            x = y0 == y1
            return -1 * s * x
        else:
            x = (a0 == a1) & (y0 == y1)
            return s * float(x)

    def mu_f(self, X, out, sensitive, y):
        expected_values_list = []
        for u in self.sensitive_classes:
            for v in self.y_classes:
                idx_true = (y == v) * (sensitive == u)  # torch.bool
                expected_values_list.append(out[idx_true].mean())
        # sensitive is star
        for v in self.y_classes:
            idx_true = y == v
            expected_values_list.append(out[idx_true].mean())
        return torch.stack(expected_values_list)

    def forward(self, X, out, sensitive, y):
        return super(EqualizedOddsLoss, self).forward(X, out, sensitive, y=y)

In [25]:
# https://towardsdatascience.com/logistic-regression-with-pytorch-3c8bbea594be
class LogisticRegression(torch.nn.Module):
     def __init__(self, input_dim, output_dim):
         super(LogisticRegression, self).__init__()
         self.linear = torch.nn.Linear(input_dim, output_dim)
     def forward(self, x):
         (self.linear) 

         outputs = torch.sigmoid(self.linear(x))
         return outputs

In [45]:
def load_compas(target_name = 'two_year_recid', sensitive_name = 'race'): 

  # Load Data 
  compas_link ='https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv'
  compas_data = pd.read_csv(compas_link, on_bad_lines='skip')

  # Trim Undesired Columns 
  compas_data = compas_data[['age' , 'juv_fel_count', 'decile_score', 'juv_misd_count', 'juv_other_count', 'priors_count','sex','c_charge_degree','race','two_year_recid']]
  compas_to_oh = ['sex','c_charge_degree']

  # Fix Race 
  compas_data.loc[compas_data.race != 'Caucasian', 'race'] = 1
  compas_data.loc[compas_data.race == 'Caucasian', 'race'] = 0

  # # Fix Race 
  # compas_data.loc[compas_data.race != 'Caucasian', 'two_year_recid'] = 1
  # compas_data.loc[compas_data.race == 'Caucasian', 'two_year_recid'] = 0

  # Get Target and Sensitive 
  target = compas_data[target_name]
  sensitive = compas_data[[sensitive_name]]

  # Remove target and one-hot-encode sex & c_charge_degree
  # compas_data = compas_data[['age' , 'juv_fel_count', 'decile_score', 'juv_misd_count', 'juv_other_count', 'priors_count','sex','c_charge_degree','race']]
  compas_data = pd.get_dummies(compas_data, columns = compas_to_oh)

  # Returns x, y, sensitive (with sensitive still included in x )
  return compas_data, sensitive_name

In [128]:
def label_encode(df):

  for i in df.columns:
    tmp = df[i].iloc[0]
    
    if type(tmp) != int or type(tmp) != float:
      le = preprocessing.LabelEncoder()
      df[i] = le.fit_transform(df[i].values)

  return df

def one_hot(df):
  
  for i in df.columns:
    tmp = df[i].iloc[0]

    if i == 'Loan_Status':
      pass
    elif type(tmp) == str:
      df = pd.get_dummies(df, columns = [i])
      # ohe = preprocessing.OneHotEncoder()
      # df[i] =  ohe.fit_transform(df[i].values.reshape(-1, 1)) # double check the shape

  return df


def get_dataset(name, target = None, sensitive = None):
  if name == 'compas':
    df, sensitive_name = load_compas()

  else: 
    df = pd.read_csv("/content/drive/MyDrive/Project/Data/adult.csv")
    df = label_encode(df)

  return df, sensitive_name

def split(x, y, sensitive_features,  train_ratio, test_ratio):
  validation_ratio = 100 - train_ratio - test_ratio

  if validation_ratio < 0:
    print ("Incorrect Ratios")
    return -1

  # train is now 75% of the entire data set
  x_train, x_test, y_train, y_test, a_train, a_test = train_test_split(x, y, sensitive_features, test_size=1 - train_ratio, random_state=32)

  # test is now 10% of the initial data set
  # validation is now 15% of the initial data set
  x_val, x_test, y_val, y_test, a_val, a_test = train_test_split(x_test, y_test, a_test, test_size=test_ratio, random_state=32) 

  return  x_train, x_test, x_val, y_train, y_test, y_val, a_train, a_test, a_val

def mae(prediction, true):
  return metrics.mean_absolute_error(prediction, true)

def accuracy(prediction, true):
  true = true.detach().numpy()
  prediction = prediction.detach().numpy()
  prediction = np.where( prediction <= 0.5, 0, 1)

  return (metrics.accuracy_score(prediction, true))

def precision(prediction, true):
  true = true.detach().numpy()
  prediction = prediction.detach().numpy()
  prediction = np.where( prediction <= 0.5, 0, 1)
  return (metrics.precision_score(prediction, true))

def recall(prediction, true):
  true = true.detach().numpy()
  prediction = prediction.detach().numpy()
  prediction = np.where( prediction <= 0.5, 0, 1)
  return (metrics.recall_score(prediction, true))

def tpr(prediction, true):
  return np.logical_and(prediction == 1, true== 1).sum()/prediction.shape[0]

def fpr(prediction, true):
  return np.logical_and(prediction == 1, true== 0).sum()/prediction.shape[0]

def tnr(prediction, true):
  return np.logical_and(prediction == 0, true== 0).sum()/prediction.shape[0]

def fnr(prediction, true):
  return np.logical_and(prediction == 0, true== 1).sum()/prediction.shape[0]


def dpr(prediction, true, sensitive_features):
  true = true.detach().numpy()
  sensitive_features = sensitive_features.detach().numpy()
  prediction = prediction.detach().numpy()
  prediction = np.where( prediction <= 0.5, 0, 1)
  return demographic_parity_ratio(true,prediction, sensitive_features=sensitive_features)

def dpd(prediction, true, sensitive_features):
  true = true.detach().numpy()
  sensitive_features = sensitive_features.detach().numpy()
  prediction = prediction.detach().numpy()
  prediction = np.where( prediction <= 0.5, 0, 1)
  return demographic_parity_difference(true,prediction, sensitive_features=sensitive_features)

def eor(prediction, true, sensitive_features):
  true = true.detach().numpy()
  sensitive_features = sensitive_features.detach().numpy()
  prediction = prediction.detach().numpy()
  prediction = np.where( prediction <= 0.5, 0, 1)
  return equalized_odds_ratio(true, prediction, sensitive_features=sensitive_features)

def eod(prediction, true, sensitive_features):
  true = true.detach().numpy()
  sensitive_features = sensitive_features.detach().numpy()
  prediction = prediction.detach().numpy()
  prediction = np.where( prediction <= 0.5, 0, 1)
  return equalized_odds_difference(true,prediction, sensitive_features=sensitive_features)
  

def test(model, x_test, y_test):
  x_test, y_test = torch.tensor(x_test), torch.tensor(y_test)
  outputs = model(x_test.float()) 
  loss = criterion(outputs, y_test.float())

  # log model results
  wandb.log({"test_loss": loss.item(), 
          "test_accuracy": accuracy(outputs, y_test), 
          "test_precision": precision(outputs, y_test), 
          "test_recall": recall(outputs, y_test)})
  wandb.watch(model)
  print ('Test Results, Loss: {:.4f},  Accuracy: {:.4f},  Precision: {:.4f},  Recall: {:.4f}' 
                  .format(loss.item(), 100 * accuracy(outputs, y_test),precision(outputs, y_test), recall(outputs, y_test)))



def train( model, criterion, optimizer, name,   lr,   x_train, x_val, y_train, y_val, a_train, a_val,sensitive,  alpha = None, regularizers = None, num_epochs = None):
  x_train, x_val, y_train, y_val, a_train, a_val = torch.from_numpy(x_train), torch.from_numpy(x_val), torch.from_numpy(y_train), torch.from_numpy(y_val), torch.from_numpy(a_train), torch.from_numpy(a_val)

  config= {
      "learning_rate":lr,
      "epochs": num_epochs,
      "model":model.__class__.__name__,
      "criterion":criterion.__class__.__name__,
      "optimizer":optimizer.__class__.__name__,
      "train_ratio":train_ratio, 
      "test_ratio":test_ratio,
      "data":name,
      "sensitive_race":sensitive
    }
  
  if regularizers != None:
    for i in range(len(regularizers)):
      config['regularizer_'+str(i)] = regularizers[i]
      config['alpha_'+str(i)] = alpha[i]

  # # store the hyperparameters in weights and bias
  wandb.init(project=name, entity="mie424",config = config)

  epoch_loss_train = []
  epoch_accuracy_train = []
  epoch_precision_train = []
  epoch_recall_train = []
  epoch_recall_dpr = []
  epoch_recall_dpd = []
  epoch_recall_eoo = []
  epoch_recall_eod = []
  
  epoch_loss_val = []
  epoch_accuracy_val = []
  epoch_precision_val = []
  epoch_recall_val = []

  # train loop
  for epoch in range(num_epochs):

    outputs = []
    # for i in range(0, x_train.shape[0], 10):
    optimizer.zero_grad()
  
    # x_i = x_train[i].float()

    # Forward pass
    outputs   = model(x_train.float()) 

      
        
    # outside of lop
    loss = criterion(outputs, y_train.float())
    if regularizers != None:
      # loss function
      for i in range(len(regularizers)):
        loss += regularizers[i](x_train.float(), outputs, a_train.float(), y_train.float()) #*alpha[i]
    # Backward and optimize
    
    loss.backward()
    optimizer.step()

    
    # store the epoch
    epoch_loss_train += [loss.item()]
    epoch_accuracy_train += [accuracy(outputs, y_train)]
    epoch_precision_train += [precision(outputs, y_train)]
    epoch_recall_train += [recall(outputs, y_train)]
    epoch_recall_dpr += [dpr(outputs, y_train.float(),a_train )]
    epoch_recall_dpd += [dpd(outputs, y_train.float(),a_train )]
    epoch_recall_eoo += [eor(outputs, y_train.float(),a_train )]
    epoch_recall_eod += [eod(outputs, y_train.float(),a_train )]

    # log model results
    wandb.log({"train_loss": epoch_loss_train[-1], 
              "train_accuracy": epoch_accuracy_train[-1], 
              "train_precision": epoch_precision_train[-1], 
              "train_recall": epoch_recall_train[-1],
              "train_dpr": epoch_recall_dpr[-1],
              "train_dpd": epoch_recall_dpd[-1],
              "train_eoo": epoch_recall_eoo[-1],
              "train_edd": epoch_recall_eod[-1]})
    
    wandb.watch(model)

    # print results
    print ('Epoch [{}/{}], Loss: {:.4f},  Accuracy: {:.4f},  Precision: {:.4f},  Recall: {:.4f}' 
                  .format(epoch+1, num_epochs, epoch_loss_train[-1], 100 * epoch_accuracy_train[-1],epoch_precision_train[-1],epoch_recall_train[-1]))
    
    # Validation loop
    with torch.no_grad():
      
      # model results
      outputs_v =  model(x_val.float()) 

      loss_v = criterion(outputs_v, y_val.float())
      if regularizers != None:
        # loss function
        for i in range(len(regularizers)):
          loss_v += regularizers[i](x_val.float(), outputs_v, a_val.float(), y_val.float()) #* alpha[i]
      

    
      # store resulst for the pass through
      epoch_loss_val += [loss_v.item()]
      epoch_accuracy_val += [accuracy(outputs_v, y_val)]
      epoch_precision_val += [precision(outputs_v, y_val)]
      epoch_recall_val += [recall(outputs_v, y_val)]
      
      # log and print model results
      wandb.log({"val_loss": epoch_loss_val[-1], 
                  "val_accuracy": epoch_accuracy_val[-1], 
                  "val_precision": epoch_precision_val[-1], 
                  "val_recall": epoch_recall_val[-1], })
      wandb.watch(model)

      print('Accuracy of validation : {} % ,  Loss: {:.4f},  Precision: {:.4f},  Recall: {:.4f}'.format(100 * epoch_accuracy_val[-1], epoch_loss_val[-1],epoch_precision_val[-1], epoch_recall_val[-1])) 

  # print ()

In [138]:
wandb.login()
# get df
name = "compas"

df, sensitive_name = get_dataset(name)

if name == "compas":
   x_var, y_var, sensitive = ['age', 'juv_fel_count', 'decile_score', 'juv_misd_count',
       'juv_other_count', 'priors_count', 'race', 'sex_Female', 'sex_Male',
       'c_charge_degree_F', 'c_charge_degree_M'], ['two_year_recid'], ['race']
else:

  x_var, y_var, sensitive = ['age', 'work_class', 'education', 'education_num',
        'marital-status', 'occupation', 'relationship', 'race', 'sex','capital_gain', 'capital_loss',
        
          'hours_per_week', 
        ]       , ['income'], ['race']
  # 'Loan_Amount_Term','LoanAmount',
name = "compas_2"

train_ratio, test_ratio = 0.7, 0.66

# select x, y, and sensitive features
x = df[x_var].values.astype(float)
y = df[y_var].values
sensitive_features = df[sensitive].values.astype(float)


# split 
x_train, x_test, x_val, y_train, y_test, y_val, a_train, a_test, a_val = split(x, y, sensitive_features, train_ratio, test_ratio)


num_epochs = 600

alpha_dp = 2000 # needs to be really high to work
DemographicParity = DemographicParityLoss(sensitive_classes=[0, 1], alpha=alpha_dp) # constraint 
alpha_eo = 0
EqualizedOdds = EqualizedOddsLoss(sensitive_classes=[0, 1], alpha=alpha_eo) # constraint 


alphas = [alpha_dp,alpha_eo]
regularizers = [DemographicParity,EqualizedOdds]
input_dim =  x_train.shape[1]# Two inputs x1 and x2 
output_dim = 1 # Single binary output 
lr = 0.0015

model = LogisticRegression(input_dim,output_dim)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

train( model, 
      criterion=criterion, 
      optimizer=optimizer, 
      name=name, 
      lr=lr, 
      x_train=x_train, 
      x_val=x_val, 
      y_train=y_train, 
      y_val=y_val, 
      a_train=a_train, 
      a_val=a_val,  
      alpha = alphas,
      sensitive =  sensitive_name,
      regularizers = None, 
      num_epochs = num_epochs)

test(model, x_test, y_test)
wandb.finish()


wandb: WARNING Serializing object of type DataFrame that is 250032 bytes


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [1/600], Loss: 4.0622,  Accuracy: 55.0208,  Precision: 0.0000,  Recall: 0.0000
Accuracy of validation : 55.84239130434783 % ,  Loss: 3.6677,  Precision: 0.0000,  Recall: 0.0000
Epoch [2/600], Loss: 3.7288,  Accuracy: 55.0208,  Precision: 0.0000,  Recall: 0.0000
Accuracy of validation : 55.84239130434783 % ,  Loss: 3.3358,  Precision: 0.0000,  Recall: 0.0000


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [3/600], Loss: 3.3960,  Accuracy: 55.0208,  Precision: 0.0000,  Recall: 0.0000
Accuracy of validation : 55.84239130434783 % ,  Loss: 3.0053,  Precision: 0.0000,  Recall: 0.0000
Epoch [4/600], Loss: 3.0646,  Accuracy: 55.0406,  Precision: 0.0004,  Recall: 1.0000
Accuracy of validation : 56.1141304347826 % ,  Loss: 2.6776,  Precision: 0.0062,  Recall: 1.0000
Epoch [5/600], Loss: 2.7359,  Accuracy: 55.1000,  Precision: 0.0018,  Recall: 1.0000
Accuracy of validation : 56.25 % ,  Loss: 2.3545,  Precision: 0.0092,  Recall: 1.0000
Epoch [6/600], Loss: 2.4119,  Accuracy: 55.1198,  Precision: 0.0031,  Recall: 0.7778
Accuracy of validation : 56.25 % ,  Loss: 2.0399,  Precision: 0.0092,  Recall: 1.0000
Epoch [7/600], Loss: 2.0965,  Accuracy: 55.2783,  Precision: 0.0070,  Recall: 0.8421
Accuracy of validation : 56.79347826086957 % ,  Loss: 1.7398,  Precision: 0.0215,  Recall: 1.0000
Epoch [8/600], Loss: 1.7961,  Accuracy: 55.5952,  Precision: 0.0154,  Recall: 0.8537
Accuracy of validation : 

test_accuracy,▁
test_loss,▁
test_precision,▁
test_recall,▁
train_accuracy,▁▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████
train_dpd,▁▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████
train_dpr,▁███████████▇█▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
train_edd,▁▄▄▅▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇██████
train_eoo,▁███████████▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▅▅▅
train_loss,█▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_precision,▁▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████


In [141]:
df['race'].unique()

array([1, 0], dtype=object)